## Demultiplexing 24nt internal bridge adaptor library

Method used to demultiplex is detailed from the following link:

https://www.protocols.io/view/sci-porec-processing-brf6m3re 

Please ask if you do not have access. Access likely restricted to members of Advanced Genomics Circuit Team

* `guppy` basecalling using 'hac' accuracy model (v4.2.2)
* Simultaneous demultiplex output based on called outer barcodes (barcode 85 for this library only)
* Creation of `lastdb` database of barcode references
* LAST aligner used to align reads to barcodes 
* Use fastx-fetcher.pl scrip to bin reads based on the assignment file

In [2]:
# ouput of error matrix file for lastal
!cat /home/minion/test_in/last/ONT_24nt_BC.mat

!echo 

# out of cat adapter fasta file for lastaldb database, barcodes 1 to 10 only but fasta holds all 96 8nt bridge adapters used
!cat /home/minion/test_in/last/ONT_24nt_BC_fwd_rev_oligo.fa

#last -Q 0
#last -a 10
#last -A 10
#last -b 5
#last -B 5
#last -S 1
# score matrix (query letters = columns, reference letters = rows):
       A      C      G      T
A      4    -24     -9    -24
C    -24      5    -24    -14
G     -9    -24      7    -24
T    -24    -14    -24      8

>ONT_BC01_fwd_rev
GATCAAGAAAGTTGTCGGTGTCTTTGTGGAATTCCACAAAGACACCGACAACTTTCTT
>ONT_BC02_fwd_rev
GATCTCGATTCCGTTTGTAGTCGTCTGTGAATTCACAGACGACTACAAACGGAATCGA
>ONT_BC03_fwd_rev
GATCGAGTCTTGTGTCCCAGTTACCAGGGAATTCCCTGGTAACTGGGACACAAGACTC
>ONT_BC04_fwd_rev
GATCTTCGGATTCTATCGTGTTTCCCTAGAATTCTAGGGAAACACGATAGAATCCGAA
>ONT_BC05_fwd_rev
GATCCTTGTCCAGGGTTTGTGTAACCTTGAATTCAAGGTTACACAAACCCTGGACAAG
>ONT_BC06_fwd_rev
GATCTTCTCGCAAAGGCAGAAAGTAGTCGAATTCGACTACTTTCTGCCTTTGCGAGAA
>ONT_BC07_fwd_rev
GATCGTGTTACCGTGGGAATGAATCCTTGAATTCAAGGATTCATTCCCACGGTAACAC
>ONT_BC08_fwd_rev
GATCTTCAGGGAACAAACCAAGTTACGTGAATTCACGTAACTTGGTTTGTTCCCTGAA
>ONT_BC09_fwd_rev
GATCAACTAGGCACAGCGAGTCTTGGTTGAATTCAACCAAGACTCGCTGTGCCTAGTT
>ONT_BC10_fwd_rev
GA

These barcodes contain from 5' to 3'
* Dpn2 site `GATC`
* Barcode in the sense orientation (24nt), italics
* EcoR1 site `GAATTC`, in bold below
* Barcode in the reverse complement (24nt), italics

e.g. 
GATC<ins>AAGAAAGTTGTCGGTGTCTTTGTG</ins>**GAATTC**<ins>CACAAAGACACCGACAACTTTCTT</ins>

Previous optimization showed that using the entire bridge adapter length gave the most sensitivity and specificity


In [13]:
# The input number of reads for barcode 85 FASTQ

%cd /media/minion/Extreme\ SSD1/iPSC_sci_poreC/24nt_run1/20201215_0826_MN23692_FAN30232_4cc1996d/hac_output

!echo "Number of reads for this library"
!seqkit stat barcode85/fastq_runid_26cdcaf2cf2fdfdd14417f2e134ebe1c3733fdc2_BC85.fastq.gz

/media/minion/Extreme SSD1/iPSC_sci_poreC/24nt_run1/20201215_0826_MN23692_FAN30232_4cc1996d/hac_output
Number of reads for this library
file                                                                          format  type   num_seqs        sum_len  min_len  avg_len  max_len
barcode85/fastq_runid_26cdcaf2cf2fdfdd14417f2e134ebe1c3733fdc2_BC85.fastq.gz  FASTQ   DNA   7,196,403  9,835,963,849       97  1,366.8   23,934


Output number of reads 7,196,403 within barcode85 directory

In [12]:
# Looking at the output barcode assignment file. 
# A read may have multiple barcodes assigned when looking into the MAF alingment file generated by LAST. 
# In this we will discard such reads instead of trying to figure which is best barcode.
# If the same barcode matchs a read multiple times, which we expect from our library this will return one line only using `uniq`
!echo "Assignment of barcodes to reads"
!gunzip -c barcode85/fastq_runid_26cdcaf2cf2fdfdd14417f2e134ebe1c3733fdc2_BC85_barcode.assignment.txt.gz | less -S | head

Assignment of barcodes to reads
ONT_BC01_fwd_rev	000026ba-c94f-417c-9e19-9a5e9b6d8e8a
ONT_BC01_fwd_rev	00004aea-536b-4486-b93b-ed77281f4a8c
ONT_BC01_fwd_rev	00006fe9-4359-4a63-a9fb-5b8562b5e611
ONT_BC01_fwd_rev	00007441-d261-4f57-a8ad-6c03e18b1386
ONT_BC01_fwd_rev	00009705-64e6-4c30-bd56-7c3772c52753
ONT_BC01_fwd_rev	0000a29d-c655-4c32-a794-a0cededc5432
ONT_BC01_fwd_rev	0000b9de-a5c2-4737-b6f5-4e553f592e47
ONT_BC01_fwd_rev	0000be58-458e-47d9-b49e-3f843df30b6c
ONT_BC01_fwd_rev	0000e296-2752-44e8-8779-d3fb9d2f57b5
ONT_BC01_fwd_rev	000121e3-f6f6-4919-9add-1cf555b5a81b


In [11]:
# Total number of barcode assignments. 
# This may not be equal to number reads assigned barcode as again a read
!echo "Number of barcodes"
!gunzip -c barcode85/fastq_runid_26cdcaf2cf2fdfdd14417f2e134ebe1c3733fdc2_BC85_barcode.assignment.txt.gz | wc -l

Number of barcodes
4689761


In [17]:
# fastx-fetcher.pl does the actual demultiplexing 
!/home/minion/fastx-fetcher.pl --help
# Bins input reads to a newly created FASTQ file for each barcode
!echo "Counts for each barcode"
!cat demultiplexed/barcode_counts.txt

perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = "en.UTF-8",
	LC_ALL = (unset),
	LC_TIME = "ja_JP.UTF-8",
	LC_MONETARY = "ja_JP.UTF-8",
	LC_ADDRESS = "ja_JP.UTF-8",
	LC_TELEPHONE = "ja_JP.UTF-8",
	LC_NAME = "ja_JP.UTF-8",
	LC_MEASUREMENT = "ja_JP.UTF-8",
	LC_IDENTIFICATION = "ja_JP.UTF-8",
	LC_NUMERIC = "ja_JP.UTF-8",
	LC_PAPER = "ja_JP.UTF-8",
	LANG = "en.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to the standard locale ("C").
Usage:
    ./fastx-fetch.pl <reads.fq> [options]

  Options:
    -help
      Only display this help message

    -idfile idFileName
      Retrieve sequence IDs from idFileName

    -quiet
      Don't report additional information to standard error

    -reverse or -v
      Invert selection logic: exclude specified sequences

    -unique
      Output only the first sequences with the same sequence ID (up to first
      space)

    -minLength len
      Only output seque

In [27]:
!echo "Number reads assigned a unique barcode"

!awk '{ sum+=$1;} END {print sum;}' demultiplexed/barcode_counts.txt

# As percentage of input
!echo
!echo "Percentage of reads assigned unique barcode"
(4288902 / 7196403) * 100

Number reads assigned a unique barcode
4288902

Percentage of reads assigned unique barcode


59.59785742960754

## Using on generated error matrix
The error matrix I used was taken from one trained by Dr David Eccles: 

David A Eccles2019.Demultiplexing Nanopore reads with LAST.protocols.io https://dx.doi.org/10.17504/protocols.io.7vmhn46 

In [28]:
!cat /home/minion/test_in/last/ONT_24nt_BC.mat

#last -Q 0
#last -a 10
#last -A 10
#last -b 5
#last -B 5
#last -S 1
# score matrix (query letters = columns, reference letters = rows):
       A      C      G      T
A      4    -24     -9    -24
C    -24      5    -24    -14
G     -9    -24      7    -24
T    -24    -14    -24      8


As our reference contains full bridge adaptor sequences and not just the ONT 24 barcode sequences it may be prudent to train our own error matrix

```
last-train --verbose -Q0 -P10 test_in/last/ONT_24nt_BC_fwd_rev_oligo.fa /media/minion/Extreme\ SSD1/iPSC_sci_poreC/24nt_run1/20201215_0826_MN23692_FAN30232_4cc1996d/hac_output/barcode85/fastq_runid_26cdcaf2cf2fdfdd14417f2e134ebe1c3733fdc2_BC85.fastq.gz > test_in/last/ONT_24nt_BC_fwd_rev_oligo.mat
```

In [33]:
# Following error-matrix file generated

!tail -n 20 /home/minion/test_in/last/ONT_24nt_BC_fwd_rev_oligo.mat

!echo

!echo "Full error-matrix output file here:"
!cat /home/minion/test_in/last/ONT_24nt_BC_fwd_rev_oligo.mat


# score matrix (query letters = columns, reference letters = rows):
#        A      C      G      T
# A    120   -547   -249   -475
# C   -582    132   -538   -449
# G   -299   -532    132   -495
# T   -535   -379   -594    117

#last -Q 0
#last -a 21
#last -A 21
#last -b 5
#last -B 6
#last -S 1
# score matrix (query letters = columns, reference letters = rows):
       A      C      G      T
A      6    -27    -12    -24
C    -29      7    -27    -22
G    -15    -27      7    -25
T    -27    -19    -30      6

Full error-matrix output file here:
# lastal version: 982
# maximum percent identity: 100
# scale of score parameters: 4.5512
# scale used while training: 91.024

# lastal -j7 -S1 -P10 -Q0 -v -r5 -q5 -a15 -b3 test_in/last/ONT_24nt_BC_fwd_rev_oligo.fa /tmp/tmpl2c9vmpp

# aligned letter pairs: 344
# deletes: 19.282
# inserts: 17.57
# delOpens: 9.372
# insOpens: 10.099
# alignments: 10
# mean delete size: 2.05741
# mean insert size: 1.73978
# delOpenProb: 0.0250273
# insOpenProb: 0

In [36]:
# Previous way on subset of data
!echo "Counts for each barcode using old error-matrix"
!cat /home/minion/test_in/barcode_counts_ONT_24nt_BC_fwd_rev_oligo.txt

!echo 
!echo "Counts for each barcode using new error-matrix"
!cat /home/minion/test_in/barcode_counts_ONT_24nt_BC_fwd_rev_oligo_2.txt

Counts for each barcode using old error-matrix
       16 ONT_BC01_fwd_rev
       16 ONT_BC02_fwd_rev
       14 ONT_BC03_fwd_rev
       16 ONT_BC04_fwd_rev
       12 ONT_BC05_fwd_rev
       20 ONT_BC06_fwd_rev
       24 ONT_BC07_fwd_rev
       12 ONT_BC08_fwd_rev
       15 ONT_BC09_fwd_rev
       15 ONT_BC10_fwd_rev
        6 ONT_BC11_fwd_rev
       13 ONT_BC12_fwd_rev

Counts for each barcode using new error-matrix
       14 ONT_BC01_fwd_rev
       15 ONT_BC02_fwd_rev
       14 ONT_BC03_fwd_rev
       16 ONT_BC04_fwd_rev
       12 ONT_BC05_fwd_rev
       19 ONT_BC06_fwd_rev
       24 ONT_BC07_fwd_rev
       11 ONT_BC08_fwd_rev
       15 ONT_BC09_fwd_rev
       15 ONT_BC10_fwd_rev
        6 ONT_BC11_fwd_rev
       14 ONT_BC12_fwd_rev


Very similar and checking each assignment there does not seem to be any changes for overlap of reads between the two methods. Rather the old matrix actually is more sensititve discovering more unique assignments

## Using minimap2 to demultiplex
Using a very similar strategy to above but using minimap2 to first align reads to barcode references. Using the same input barcode reference fasta. However, this requires a few adaptations to the default parameters for `map-ont` preset in `minimap2`. Using the following guidance from this paper that also used minimap2 to demultiplex reads: 

https://www.microbiologyresearch.org/content/journal/mgen/10.1099/mgen.0.000336?crawler=true 

```
"The ONT native barcoding kit was used to multiplex the 12 pools (NB01–NB12), each containing eight sets of pooled amplicons (PB01–PB08), resulting in 96 dual-barcodes (listed in Supplementary file 1) for the 96 samples. Please note that our PCR barcodes were designed based on the sequences of native barcodes. After initial examination of sequencing reads, we found that the native barcode kit attached the reverse complement barcode sequence along with a sequence of ‘CAGCACCT’ to the 5′ end of amplicons. The 96 dual-barcode sequences were generated using generatebcs.py [e.g. NB01PB01: rc(NB01)+CAGCACCT+(PB01)]. We mapped the sequencing reads to the dual-barcode sequences with 56 bp in length using Minimap2 (v2.11) [26] by -k7 -A1 -m42 -w1 options. With a pairwise read mapping format (PAF) file outputted by Minimap2, we generated 96 sequencing files (e.g. NB01PB01.fq) to include the corresponding sequencing reads for a specific dual-barcode using getbcfq.py."
```

` -k7 -A1 -m42 -w1`

Run following command:
```
minimap2 -k7 -A1 -m42 -w1 test_in/last/ONT_24nt_BC_fwd_rev_oligo.fa test_in/fastq_runid_26cdcaf2cf2fdfdd14417f2e134ebe1c3733fdc2_0_0.fastq.gz > test_in/ONT_24nt_BC_fwd_rev_oligo_test.paf
```

In [ ]:
# Number of assigned reads was few in the tens. So does not seem like good strategy

!less -S /home/minion/test_in/ONT_24nt_BC_fwd_rev_oligo_test.paf

713b1acd-cc89-4d44-9f31-1361c4e40487    2123    303     353     +       ONT_BC07
713b1acd-cc89-4d44-9f31-1361c4e40487    2123    307     353     -       ONT_BC07
846faa8f-0d9a-4733-884d-e7433fe84188    1291    414     470     +       ONT_BC06
4594342c-7917-40c2-ae9b-952ca1dbc22a    843     126     176     +       ONT_BC07
069469cf-1a4e-4827-91c7-89cda8e1024f    1316    1060    1118    +       ONT_BC01
069469cf-1a4e-4827-91c7-89cda8e1024f    1316    1064    1122    -       ONT_BC01
8b2bbc9e-9a95-4222-980b-d7e6422474f5    1035    313     361     +       ONT_BC07
8b2bbc9e-9a95-4222-980b-d7e6422474f5    1035    317     361     -       ONT_BC07
5543396c-0b67-4519-92e4-29907d5f50a6    1164    1015    1061    +       ONT_BC07
81016a30-761b-4588-9f35-89d22ece0980    823     237     294     +       ONT_BC05
81016a30-761b-4588-9f35-89d22ece0980    823     241     294     -       ONT_BC05
6c184135-4126-4f47-b850-63026147e44e    705     598     655     +       ONT_BC04
6c184135-4126-4f47-b850-6302